In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
track = pd.read_excel('./baliky.xlsx')

In [ ]:
cpost = dict(zip(track[track.provider == 'posta'].tracking, track[track.provider == 'posta'].route))

In [ ]:
data = pd.DataFrame()

for tr in cpost:
    print(tr)
    r = requests.get('https://www.postaonline.cz/trackandtrace/-/zasilka/cislo?parcelNumbers=+' + tr + '+')
    soup = BeautifulSoup(r.text, 'html.parser')
    for row in soup.find_all('table', class_='datatable2')[1].find_all('tr'):
        recs = row.find_all('td')
        if len(recs) > 0:
            try:
                url = recs[3].find('a').get('href')
            except:
                url = None
            data = data.append({
                'tracking': tr,
                'date': recs[0].text, 
                'status': recs[1].text, 
                'psc': recs[2].text,
                'place_name': recs[3].text,
                'place_url': url
            },ignore_index=True)

In [ ]:
data['addr'] = None

In [ ]:
for i, row in data.iterrows():
    if row[2] != None:
        r = requests.get(row[2])
        soup = BeautifulSoup(r.text, 'html.parser')
        addr = soup.find('div', class_='infoCardContent').find('div', class_='span12').find('p')
        data.ix[i].addr = addr

In [ ]:
def cln(val):
    val = str(val)
    if val == val:
        return val.replace('<p>', '').replace('</p>', '').replace('<br/>', ', ')
    else:
        return None

data.addr = data.addr.apply(lambda x: cln(x))

In [ ]:
data.to_excel('./tracking.xlsx', index=False)